In [ ]:
# First, ensure you have the necessary libraries installed
! pip install -qU google-generativeai==0.8.5 langgraph langchain langchain-google-genai beautifulsoup4 requests==2.32.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 587.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 10.7 MB/s eta 0:00:00


In [ ]:

import os
import getpass
import re
from typing import TypedDict, List
import requests
from bs4 import BeautifulSoup
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage




In [ ]:
# --- 1. Setup Gemini LLM ---
# It's better to set the API key as an environment variable,
# but getpass is fine for testing.
try:
    # Use a cached API key if it exists to avoid re-entering
    if "GOOGLE_API_KEY" not in os.environ:
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Gemini API key: ")
except (ImportError, NameError):
    print("Could not get API key securely. Please set the GOOGLE_API_KEY environment variable.")
    # Add a fallback for non-interactive environments
    if "GOOGLE_API_KEY" not in os.environ:
        os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE"


Enter your Gemini API key: ··········


In [ ]:


llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0)

In [ ]:
# --- 2. Define the State ---
class AgentState(TypedDict):
    """Defines the structure of the data passed between nodes."""
    user_input: str
    product_details: str
    analysis_result: str
    recommendation: str

In [ ]:
# --- 3. Define Graph Nodes ---
def get_user_input(state: AgentState) -> AgentState:
    """Gets the product URL or name from the user."""
    user_input = input("Welcome to Conscious Cart! 🛒 Please enter a product URL or a product name: ")
    state["user_input"] = user_input
    return state

def scrape_product_details(state: AgentState) -> AgentState:
    """
    Scrapes a product page. If it fails, it falls back to analyzing the URL itself.
    """
    url = state["user_input"]
    print("--- 🕵️ Fetching product data from URL... ---")
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        }
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.find('span', id='productTitle')
        features = soup.find('div', id='feature-bullets')

        title_text = title.get_text(strip=True) if title else "Title not found"
        features_text = features.get_text(strip=True) if features else "Features not found"

        if title_text == "Title not found" and features_text == "Features not found":
            raise ValueError("Failed to find key product details.")

        state["product_details"] = f"Product Title: {title_text}\n\nProduct Features:\n{features_text}"

    except Exception as e:
        print(f"\n---  scraping failed: {e} ---")
        print("---  fallback to url analysis ---")
        try:
            match = re.search(r'/([^/]+)/dp/|/([^/]+)$', url)
            if match:
                url_text = (match.group(1) or match.group(2)).replace('-', ' ').replace('+', ' ')
                state["product_details"] = (
                    "NOTE: The product page was inaccessible. The following analysis is based on keywords from the URL.\n\n"
                    f"Keywords from URL: {url_text}"
                )
            else:
                state["product_details"] = "" # Set to empty if fallback also fails
        except Exception:
            state["product_details"] = ""

    return state

def generate_details_from_name(state: AgentState) -> AgentState:
    """Generates a plausible product description from a product name."""
    product_name = state["user_input"]
    print(f"--- 📝 Generating hypothetical details for: {product_name} ---")
    prompt = (
        f"Generate a typical e-commerce product description for '{product_name}'. "
        "Include likely materials and features for an environmental impact analysis."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["product_details"] = response.content.strip()
    return state

def analyze_environmental_impact(state: AgentState) -> AgentState:
    """Analyzes the product details for environmental impact."""
    details = state["product_details"]
    if not details:
        state["analysis_result"] = "ANALYSIS_FAILED"
        state["recommendation"] = "Could not generate a recommendation as no product data was found."
        return state

    print("--- 🔬 Analyzing environmental impact... ---")
    prompt = (
        "You are an environmental analyst. Based on the details below, provide a brief impact analysis. "
        "Conclude with a verdict on its own line: 'RECOMMENDED', 'NOT RECOMMENDED', or 'NEUTRAL'.\n\n"
        f"Product Details:\n---\n{details}\n---"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["analysis_result"] = response.content.strip()
    return state

def generate_recommendation(state: AgentState) -> AgentState:
    """Generates the final recommendation, finding alternatives if necessary."""
    analysis = state["analysis_result"]
    if "ANALYSIS_FAILED" in analysis:
        return state # Recommendation already set in the analysis node

    print("--- ✨ Generating final recommendation... ---")
    if "NOT RECOMMENDED" in analysis.upper():
        prompt = (
            "The following product was 'Not Recommended'. Suggest 3 specific, sustainable alternatives. "
            "For each, briefly explain why it's a better choice. Format as a numbered list.\n\n"
            f"Original Product Analysis:\n---\n{analysis}\n---"
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        alternatives_text = response.content.strip()
        state["recommendation"] = (
            "## Product Analysis 📉\n\n"
            f"{analysis}\n\n"
            "## Better, Eco-Friendly Alternatives 🌱\n\n"
            f"{alternatives_text}"
        )
    else:
        state["recommendation"] = f"## Product Analysis ✅\n\n{analysis}"

    return state

In [ ]:
# --- 4. Define Graph Routers ---
def route_by_input_type(state: AgentState) -> str:
    """Routes the process based on whether the input is a URL or a product name."""
    if state["user_input"].lower().startswith("http"):
        return "scrape_details"
    return "generate_from_name"

In [ ]:
# --- 5. Build the Graph ---
builder = StateGraph(AgentState)

builder.add_node("get_input", get_user_input)
builder.add_node("scrape_details", scrape_product_details)
builder.add_node("generate_from_name", generate_details_from_name)
builder.add_node("analyze_impact", analyze_environmental_impact)
builder.add_node("create_recommendation", generate_recommendation)

builder.set_entry_point("get_input")
builder.add_conditional_edges("get_input", route_by_input_type)

builder.add_edge("scrape_details", "analyze_impact")
builder.add_edge("generate_from_name", "analyze_impact")
builder.add_edge("analyze_impact", "create_recommendation")
builder.add_edge("create_recommendation", END)

graph = builder.compile()


In [ ]:
# --- 6. Run the Agent ---
final_state = graph.invoke({})

# Print the final, user-facing recommendation
print("\n\n===================================")
print("      Final Recommendation")
print("===================================\n")
print(final_state.get("recommendation", "No recommendation could be generated."))

Welcome to Conscious Cart! 🛒 Please enter a product URL or a product name: Product Details Name : GHAR SOAPS MAGIC SOAP 100 GM  Product Name : GHAR SOAPS MAGIC SOAP 100 GM  Brand Name : Ghar Soaps  Type : Solid  Flavour : Sandalwood  Net Quantity (N) : 1  PROVEN TO BRIGHTEN THE SKIN  HOW DOES THIS SOAP MAKES YOUR SKIN BRIGHTER ?     Saffron present in the soap makes the skin brighter. Saffron helps in deeply exfoliating the pores giving you a cleansed, fresh and flawless skin. Scars, sun spots, dark patches and blemishes fades quickly.     Our soaps are completely hand made and ingredients we use are high quality and we make soaps without harmful cheaper chemicals. 98% of soaps available in market are made from harmful chemicals.     Also helps in reducing the pore size of the skin     Lightens and diminishes the appearance of scars and blemishes.  Crafted to provide you the ultimate luxurious bathing experience  Cruelty-free and Clear Conscience  Free of Parabens, Sulphates and Minera